My very first example of understanding and learning Decision trees. And an  attempt to code the logic for this simple example in python.

https://medium.com/deep-math-machine-learning-ai/chapter-4-decision-trees-algorithms-b93975f7a1f1

https://www.kaggle.com/pragyanbo/decision-trees-from-scratch

In [ ]:
import numpy as np
import pandas as pd
import math

###1. Creating the Demo dataset.

In [ ]:
# The initial dataset 
arr = np.array([['sunny','sunny','overcast','rainy','rainy','rainy','overcast','sunny','sunny','rainy','sunny','overcast',
                  'overcast','rainy'],
                ['hot','hot','hot','mild','cool','cool','cool','mild','cool','mild','mild','mild','hot','mild'],
                ['high','high','high','high','normal','normal','normal','high','normal','normal','normal','high',
                 'normal','high'],
                ['false','true','false','false','false','true','true','false','false','false','true','true','false','true'],
                ['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']]) 

# converting out 2D array into a dataframe with named columns 
df = pd.DataFrame(arr.T)
df.columns = ['Outlook','Temp','Humidity','Windy','Play']

print(df)

# splitting the data into features(X) and target(y)
df_x = df.iloc[:,:-1]
y = pd.DataFrame(df.iloc[:,-1])

     Outlook  Temp Humidity  Windy Play
0      sunny   hot     high  false   no
1      sunny   hot     high   true   no
2   overcast   hot     high  false  yes
3      rainy  mild     high  false  yes
4      rainy  cool   normal  false  yes
5      rainy  cool   normal   true   no
6   overcast  cool   normal   true  yes
7      sunny  mild     high  false   no
8      sunny  cool   normal  false  yes
9      rainy  mild   normal  false  yes
10     sunny  mild   normal   true  yes
11  overcast  mild     high   true  yes
12  overcast   hot   normal  false  yes
13     rainy  mild     high   true   no


###2. Entropy calculation of the dataset. 
First task, when given a dataset is to calculate the entropy of the entire dataset.So, we write a function of entropy calculation.

**Entropy**\
Entropy H(S) is a measure of the amount of uncertainity in the data set (S) (i.e entropy characterizes the (data)).

\begin{align}
 H(s) = \sum_{c\in C} -p(c) log_2 p(c)
\end{align}

where, \
S - The current data set for which entropy is being calculated (changes in every iteration)\
C - Set of classes in S = {yes,no}\
p(c) - The proportion of the number of the number of elements in class c to the no elements in set S.\
When $H(S)=0$, the set $S$ is perfectly classified (i.e all elements in $S$ are of the same class).\

In $ID3$, entropy is calculated for each remaining attribute. The attribute with the smallest entropy is used to splot the set $S$ on their iteration. The higher the entropy, the higher the potential to improve the classification here.


###3. To select our first node 

To crete a tree we need to have a root node first and we know that nodes are features/attributes ( Outlook,Temp,Humidity & windy).

Q. So which one do we pick first ?\
Determine the attirbute that best classifies the training data; use this attribute at the root of the tree. Repeat this process for each branch. This mean, we are performning top-down greedy search through the space of possible decision trees.

Q. How do we choose the best attribute ? \
Use the attribute with the highest "Information gain" in ID3.

**Information Gain**

Information gain IG(A) is the measure of the difference in entropy from before to after the set $S$ in split on an attribute $A$. In other words. how much uncertainity in $S$ was reduced after splitting set $S$ on attribute $A$. 

\begin{align}
IG(A,S) = H(S) - \sum_{t\in T} p(t) H(t)
\end{align}

where,\
$H(S)$ : Entropy of set $S$.\
$T$ : The subsets created from splitting set $S$ by attribute $A$ such that $S = \cup_{t \in T}t$\
$p(t)$ : The proportion of the number of elements in $t$ to the no of elements in set $S$.
$H(t)$ : Entropy of subset $t$.

In ID3, Information Gain can be calculated (instead of entropy) for each remaining attribute. The attribute with the **largest Information Gain** is used to split the set $S$ on this iteration.

In [ ]:
'''First task, when given a dataset is to calculate the entropy of the entire dataset.So, we write a function of entropy calculation '''
def entropy(df,y):
  # Entropy of the entire dataset   
  ''' comments to understand the implementation in first pass'''
  # define the variable entropy to hold th value of the calculated entropy
  entropy = 0
  class_val = y.value_counts()
  for i in range(len(class_val)):
    entropy += - (np.divide(class_val[i],np.sum(class_val))) * math.log(np.divide(class_val[i],np.sum(class_val)),2)
  return entropy 

In [ ]:
'''Updating the entropy function for each attribute w,r,t 'y' and then pass the result to information gain function'''

def entropy_features(df,y,cols):
  # Using the groupby function to group our feature vector subclasses with target class
  # define the list entropy to hold the values of the calculated entropies of all the viable features
  avg_entropy = []
  avg_cal = 0

  for k in range(0,df.shape[1]-1):
  
    unq_attr = df.iloc[:,k].unique()
    f = df.groupby([cols[k],cols[-1]]).size().reset_index(name="counts")
    g = df.iloc[:,k].value_counts().reset_index(name="counts")
    avg_cal = 0
    #print("We are here in enotrpy_features")

    for i in range(0,len(unq_attr)):
      cal = 0 
      for j in range(0,len(f)):

        if f.iloc[j,0] == g.iloc[i,0]:
          cal += - (np.divide(f.iloc[j,-1],g.iloc[i,1])) * math.log(np.divide(f.iloc[j,-1],g.iloc[i,-1]),2)
      avg_cal += np.divide(g.iloc[i,-1],len(df)) * cal
    avg_entropy.append(avg_cal)
  return avg_entropy
 

In [ ]:
def information_gain(entropy_l,entropy_d,cols):

  # entropy_d : the entropy of the target column 
  # entropy_l : list of all average entropy of all the features considered for this subset of the dataset.
  node= np.argmax(entropy_d - entropy_l)
  return cols[node],node 

In [ ]:
def sub_dataframe(df,y,cols,node,node_attr):
  
    sub_df = df[df.iloc[:,node]==node_attr]
    x = sub_df.drop(columns=cols[node])
    y = sub_df.iloc[:,-1]
    return x,y

In [ ]:
def build_tree(df,y,tree):

  # Define the tree dictionary, to hold the tree structure, as we recursively traverse the tree.
  cols = df.columns
 
  # Number of unique target values 
  entropy_ds = entropy(df,y) 
 
  # get the avg entropy of all features of the df and get the best fit, using information gain function
  entropy_l = entropy_features(df,y,cols)
  feature, node = information_gain(entropy_l,entropy_ds,cols)
 
  tree = {}
  tree.update({feature:{}})

  #get the number of unique attributes in the feature. 
  node_attr = df.iloc[:,node].unique()

  
  for key in range(0,len(node_attr)):

    tree[feature].update({node_attr[key]:""})
    # get the sub_dataFrame w.r.t to the attribute selected 
    sub_df,y = sub_dataframe(df,y,cols,node,node_attr[key])
    count = y.nunique()
    
    # check the counts with respect to target 
    if count == 1:
      tree[feature][node_attr[key]] = y.iloc[0]
    else: 
      tree[feature][node_attr[key]] = build_tree(sub_df,y,tree)
  return tree


In [ ]:
tree = {}
build_tree(df,y,tree)

{'Outlook': {'overcast': 'yes',
  'rainy': {'Windy': {'false': 'yes', 'true': 'no'}},
  'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}}}

To do :
1. Try to update the build tree. to accomodate max depth. nd implement with another example. load_breast data 
2. With the help of pydot.. plot the tree. 

3. Regression tree (CART) : Gini Index 

In [ ]:
cols = df.columns
attr = df.iloc[:,0].unique()
print(cols)
print(attr)

tree = {}
tree['Outlook'] = {attr[0]:""}
tree['Outlook'].update({attr[1]:""})
tree['Outlook'].update({attr[2]:""})


print(tree)

Index(['Outlook', 'Temp', 'Humidity', 'Windy', 'Play'], dtype='object')
['sunny' 'overcast' 'rainy']
{'Outlook': {'sunny': '', 'overcast': '', 'rainy': ''}}
{'Outlook': {'sunny': 0, 'overcast': '', 'rainy': ''}}
